In [37]:
import os
import shutil
import pyspark
from pyspark.sql import SparkSession

In [38]:
pyspark.__version__

'2.4.4'

In [78]:
spark = (SparkSession
         .builder
         .config('spark.driver.memory', '1000g')
         .config('hive.exec.orc.default.stripe.size', 1024*1024*1024)
         .getOrCreate()
         )

In [79]:
data_dir = '/mnt/isilon1/data/mortgage'
perf_file = os.path.join(data_dir, 'perf/Performance_*.txt')
# perf_file = os.path.join(data_dir, 'perf/Performance_2016Q1.txt')
print(perf_file)

/mnt/isilon1/data/mortgage/perf/Performance_*.txt


In [80]:
dtypes = [
    ("loan_id", "bigint"),
    ("monthly_reporting_period", "date"),
    ("servicer", "string"),
    ("interest_rate", "double"),
    ("current_actual_upb", "double"),
    ("loan_age", "double"),
    ("remaining_months_to_legal_maturity", "double"),
    ("adj_remaining_months_to_maturity", "double"),
    ("maturity_date", "string"),
    ("msa", "double"),
    ("current_loan_delinquency_status", "int"),
    ("mod_flag", "string"),
    ("zero_balance_code", "string"),
    ("zero_balance_effective_date", "date"),
    ("last_paid_installment_date", "date"),
    ("foreclosed_after", "date"),
    ("disposition_date", "date"),
    ("foreclosure_costs", "double"),
    ("prop_preservation_and_repair_costs", "double"),
    ("asset_recovery_costs", "double"),
    ("misc_holding_expenses", "double"),
    ("holding_taxes", "double"),
    ("net_sale_proceeds", "double"),
    ("credit_enhancement_proceeds", "double"),
    ("repurchase_make_whole_proceeds", "double"),
    ("other_foreclosure_proceeds", "double"),
    ("non_interest_bearing_upb", "double"),
    ("principal_forgiveness_upb", "double"),
    ("repurchase_make_whole_proceeds_flag", "string"),
    ("foreclosure_principal_write_off_amount", "double"),
    ("servicing_activity_indicator", "string")
]

In [81]:
schema = ','.join([' '.join(col) for col in dtypes])

In [82]:
df = spark.read.load(perf_file, format='csv', sep='|', schema=schema, header=False, dateFormat='MM/dd/yyyy')

In [83]:
size_multiplier = 1.0
if size_multiplier != 1.0:
    print('Resampling')
    df = df.sample(True, size_multiplier, seed=7)

In [84]:
df = df.repartition(48)

In [85]:
orc_dir = os.path.join(data_dir, 'from-spark3.orc')
if os.path.exists(orc_dir): shutil.rmtree(orc_dir)
(df.write
 .format('orc')
 .option('compression','snappy')
 .save(orc_dir)
)